In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense


모델설계

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None, 50))
encoder_lstm = LSTM(256, return_state=True) # LSTM 레이어 사용

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs: 층의 타임 스텝에서의 출력 정보(출력 상태)
# state_h, state_c: hidden state, cell state

encoder_states = [state_h, state_c]
 # 디코더에 전달될 인코더 정보(최종 상태) state_h , state_c



# 인코더
# 입력 시퀀스 -> hidden state (고정된 크기의 컨텍스트 벡터)  변환
# hidden state 벡터 : 입력 데이터의 전체 정보 압축 표현(일정 크기)
# RNN, LSTM, GRU 구조 사용

In [ ]:
# 디코더
decoder_inputs = Input(shape=(None, 50))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
   # 인코더로부터 전달받은 정보를 (encoder_states : [state_h, state_c])를 초기 state로 사용
decoder_dense = Dense(50, activation='softmax')
   # 출력 클래스(50개 ex, 다음 단어)에 대한 확률 분포 생성
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# 모델 컴파일
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 요약
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, None, 50)]           0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, None, 50)]           0         []                            
                                                                                                  
 lstm_2 (LSTM)               [(None, 256),                314368    ['input_5[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                            

모델학습

In [ ]:
# 입력, 출력 데이터 생성 (예시 데이터)
encoder_input_data = np.random.rand(10000, 10, 50)  # 10000개 데이터(길이 10의 시퀀스, 각 시퀀스는 50차원 벡터)
decoder_input_data = np.random.rand(10000, 10, 50)
decoder_target_data = np.random.rand(10000, 10, 50)


In [ ]:
encoder_input_data[0]

In [ ]:
# 모델 학습
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
125/125 [==============================] - 21s 135ms/step - loss: 118.6560 - accuracy: 0.0202 - val_loss: 120.1655 - val_accuracy: 0.0195
Epoch 2/10
125/125 [==============================] - 16s 129ms/step - loss: 120.2139 - accuracy: 0.0198 - val_loss: 120.0013 - val_accuracy: 0.0195
Epoch 3/10
125/125 [==============================] - 17s 133ms/step - loss: 119.9472 - accuracy: 0.0198 - val_loss: 120.0142 - val_accuracy: 0.0195
Epoch 4/10
125/125 [==============================] - 17s 134ms/step - loss: 120.0157 - accuracy: 0.0198 - val_loss: 119.9015 - val_accuracy: 0.0195
Epoch 5/10
125/125 [==============================] - 15s 122ms/step - loss: 119.9717 - accuracy: 0.0198 - val_loss: 120.0202 - val_accuracy: 0.0195
Epoch 6/10
125/125 [==============================] - 15s 117ms/step - loss: 119.8603 - accuracy: 0.0198 - val_loss: 120.1582 - val_accuracy: 0.0195
Epoch 7/10
125/125 [==============================] - 15s 120ms/step - loss: 119.9594 - accuracy: 0.0198 -

In [ ]:
# 모델 평가
# 학습된 모델을 평가하기 위해 테스트 데이터를 사용
encoder_input_test = np.random.rand(2000, 10, 50)
decoder_input_test = np.random.rand(2000, 10, 50)
decoder_target_test = np.random.rand(2000, 10, 50)

loss, accuracy = model.evaluate([encoder_input_test, decoder_input_test], decoder_target_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


63/63 [==============================] - 2s 27ms/step - loss: 120.1078 - accuracy: 0.0220
Test Loss: 120.10777282714844, Test Accuracy: 0.02199999988079071


예측 수행(모델 적용)

In [ ]:
# 인코더 - 새로운 입력 시퀀스를 받아 hidden state 계산
encoder_model = Model(encoder_inputs, encoder_states)
 # encoder_inputs: 이전에 정의된, 훈련된 모델 (변수명 model)의 입력층
 # encoder_states: hidden state, cell state (LSTM)

# 원본 model에서 LSTM 등의 레이어를 통해 학습된 가중치를 새로운 모델의 구성 요소로 재사용
# 학습 단계에서 얻은 데이터의 표현(특징)을 추론 단계에서 활용

In [ ]:
# 디코더 입력
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [ ]:
# 디코더 출력
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:


# 디코더 모델 정의
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 시퀀스 종료 토큰과 최대 디코딩 시퀀스 길이 정의
eos_token_index = 0  # 예시로 0번 인덱스를 종료 토큰으로 사용
max_decoder_seq_length = 10

In [ ]:

def decode_sequence(input_seq):
    # 인코더에서 상태 벡터 얻기
    states_value = encoder_model.predict(input_seq)

    # 디코더 입력 준비 (시퀀스 시작 토큰)
    target_seq = np.zeros((1, 1, 50))

    # 시퀀스 종료 조건
    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 예측된 토큰을 디코딩하여 출력 시퀀스에 추가
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = sampled_token_index  # 예시에서는 인덱스 자체를 토큰으로 사용

        decoded_sentence.append(sampled_token)

        # 시퀀스 종료 조건
        if (sampled_token == eos_token_index or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 디코더 입력 업데이트 (다음 입력 토큰)
        target_seq = np.zeros((1, 1, 50))
        target_seq[0, 0, sampled_token] = 1.0

        # 상태 업데이트
        states_value = [h, c]

    return decoded_sentence



In [ ]:
# 새로운 입력 시퀀스 (예시 데이터)
new_encoder_input = np.random.rand(1, 10, 50)




In [ ]:
# 예측된 출력 시퀀스
decoded_sentence = decode_sequence(new_encoder_input)
print('Predicted sequence:', decoded_sentence)